# TriScale - The experiment design and analysis framework

**To be written once we are done**


Some random tests

| **Author** |**Last modified**|
|:---:|:---:|
|Romain Jacob  |22-08-2019|

This notebook presents the _TriScale_ framework. _TriScale_ is composed of four modules, called
* network profiling
* experimental design
* preprocessing
* analysis.

These modules support the experimenter at different stages of experimental campaigns: **before** (_network profiling_ and _experimental design_), **during** (_preprocessing_) and **after** (_analysis_).
This notebook describe in details the functionalities provided by each of these modules, using a real experimental campain as illustration.


We start by importing the _etalon_ module, which contains the implementation of the entire framework.

In [3]:
# Standard library imports
from pathlib import Path

import numpy as np

import triscale

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
print(plotly.__version__)

4.0.0


In [39]:
import triplots
import plotly.graph_objects as go

layout = {'title':'prout', 'width':200}
layout = None
x = [6,15,156,3,1,7,6,1,2,79,21,3,4,5]
triplots.autocorr_plot(x, layout=layout)
triplots.autocorr_plot(x, layout=layout)

# ---------------------------------------------------------------- 
# TODO autocorr_plot
# ---------------------------------------------------------------- 
- polish the plot: display tol and conf in the legend
- check the custom layout input
- finish the docstring (input/output, support to save the plot)
# ---------------------------------------------------------------- 



## Network profiling


**TL;DR**  
- Evaluates whether the network used for experiment can be consider 'stable' enough to enable repeatable results.
- If so, returns 
  - the (minimal) time interval over which the runs should be performed
  - the variability measure of that network (see Analysis)

**Whether or not we want to do the network profiling with Pantheon is still under consideration...**

In [106]:
import UseCase_Glossy.flocklab as flocklab

data_file = 'ExampleTraces/2019-07_FlockLab_dpp-cc430.csv'
data_file = 'ExampleTraces/2019-07_FlockLab_sky.csv'
df = flocklab.parse_data_file(data_file, active_link_threshold=50)


# ---------------------------------------------------------------- 
# TODO parse_data_file 
# ---------------------------------------------------------------- 
- write a doctring
- input check
- iterate through the list of inputs to see if we have all we need
# ---------------------------------------------------------------- 



In [108]:
link_quality_bounds = [0,100]
link_quality_name = 'PRR [%]'
results = triscale.network_profiling(df, link_quality_bounds, link_quality_name, print_output=True)

# ---------------------------------------------------------------- 
# TODO network_profiling 
# ---------------------------------------------------------------- 
- verbose display (clearly) advising people what they should do
i.e., how long should be the time span of an experiment
- write a doctring
- write output to file
- input check
- iterate through the list of inputs to see if we have all we need
- handle the intermediary outputs
- format the final output
- add a test on the minimal number of samples required to be left in the autocorellation
# ---------------------------------------------------------------- 

# ---------------------------------------------------------------- 
# TODO theil_plot
# ---------------------------------------------------------------- 
- check the custom layout input
- write the docstring (input/output, support to save the plot)
- uniformize the plot colors (use the same as TriScale logo)
# ---------------------------------------------------------------- 

# ---------------------------------------------------------------- 
# TODO autocorr_plot
# ---------------------------------------------------------------- 
- check the custom layout input
- finish the docstring (input/output, support to save the plot)
- uniformize the plot colors (use the same as TriScale logo)
# ---------------------------------------------------------------- 



# ---------------------------------------------------------------- 
# TriScale report - Network profiling
# ---------------------------------------------------------------- 

Network 	FlockLab - DPP-cc430

Profiling time span
from 		2019-07-09 23:00:02+00:00
to 		2019-08-19 21:00:02+00:00

Profiling granularity
		0 days 02:00:00

# ---------------------------------------------------------------- 

Network link quality does NOT appears I.D.D. !
Searching for a suitable time interval...

Window size: 2	Stationary: 0
Window size: 3	Stationary: 0
Window size: 4	Stationary: 0
Window size: 5	Stationary: 0
Window size: 6	Stationary: 0
Window size: 7	Stationary: 1


With a confidence of 95%
network link quality appears stationary over a 
time span of	0 days 14:00:00

# ---------------------------------------------------------------- 



## Experimental design

**TL;DR**  
Determine the minimal number of samples (i.e., runs) required to estimate a given distribution percentile with the given level of confidence.

In [185]:
# perc = 50
perc=50
alpha = 75

triscale.experiment_design(perc,alpha)
triscale.experiment_design(perc,alpha,robustness=2)
# triscale.experiment_design(perc,alpha,nb_samples=20)


# ---------------------------------------------------------------- 
# TODO 
# ---------------------------------------------------------------- 
- account for the double-sided/single-sided case
- verbose display (clearly) advising people what they should do
i.e., how many samples in a data serie must be collected
- write a doctring
- write output to file (?) I don't think that's necessary here...
- iterate through the list of inputs to see if we have all we need
- adapt to make ci() return a dictionaty
- rename ci() -> ci_thompson? thompson_ci? 
# ---------------------------------------------------------------- 

2 3
# ---------------------------------------------------------------- 
# TODO 
# ---------------------------------------------------------------- 
- account for the double-sided/single-sided case
- verbose display (clearly) advising people what they should do
i.e., how many samples in a data serie must be collected
- write a doctring
- write output to file (?) I don't think th

## Preprocessing

**TL;DR**  
Estimate whether the dynamics of the protocol under test have converged by the end of a run. If not, this run cannot be used to estimate the steady-state performance of the protocol.

In [60]:
# This is a simple example to show the case where the serie's average converges
# but oscillate in an unstable manner

data_file_name = 'ExampleTraces/diverging_trace.csv'
metric = {'name': 'x * sin(x)',
          'measure': 95}
convergence = {'expected': True,
               'confidence': 95,  # in %
               'tolerance': 1,    # in %
              }
triscale.analysis_preprocessing(data_file_name, 
                                metric, 
                                plot=True,
                                convergence=convergence,
                                verbose=True)

# ---------------------------------------------------------------- 
# TODO 
# ---------------------------------------------------------------- 
- modify the ploting: it should plot regardless of whether the convergence test is run or not
-> Take the theil_plot function out of the convergence test and put in here directly. Adapt to show the regression and bounds only when convergence is expected

- write the doctring
- modif convergence_test() to output a dictionary
- check for crazy values in the input dictionaries
- polish the plot: display tol and conf in the legend
# ---------------------------------------------------------------- 

/!\ Non-stationary /!\
The 95% CI on the link quality trend exceeds the tolerance.
95% CI(scaled): 	[-0.23977472902182315 , 0.2131588514349058]
Tolerance: 		[-0.01 , 0.01]

[ FAILED ]
With a confidence level of 	95%
given a tolerance of 		1%
Run has NOT converged.

[ WARNING ] These data should not be used to estimate 
the long-term performance of the sy

(False, nan)

## Analysis


**TL;DR**  
Perform the entire experiment analysis, that is:
- Assess the independance of runs
- Compute the performance report for the protocol under test
- If multiple series were performed, assess the repeatability of the protocol under test.

### Preprocessing

In [84]:
data_path = Path('ExampleTraces')
data_files = [
    'vegas_datalink_throughput_run1_flow1_egress.csv',
#     'vegas_datalink_throughput_run1_flow1_ingress.csv',
#     'vegas_datalink_delay_run1_flow1.csv'
]

metric = {'name': 'Throuput',
          'unit': 'MBit/s',
          'measure': 95,
          'bounds': [0,120]}
convergence = {'expected': False,
               'confidence': 95,  # in %
               'tolerance': 1,    # in %
              }

series_metric = []
for file in data_files:
    data_file_name = str(data_path / file)
    output = triscale.analysis_preprocessing(data_file_name, 
                                           metric, 
                                           plot=True, 
                                           convergence=convergence, 
                                           verbose=True,
                                           plot_out_name=None)
    series_metric.append(output)
    
print(series_metric)


# ---------------------------------------------------------------- 
# TODO 
# ---------------------------------------------------------------- 
- modify the ploting: it should plot regardless of whether the convergence test is run or not
-> Take the theil_plot function out of the convergence test and put in here directly. Adapt to show the regression and bounds only when convergence is expected

- write the doctring
- modif convergence_test() to output a dictionary
- check for crazy values in the input dictionaries
- polish the plot: display tol and conf in the legend
# ---------------------------------------------------------------- 

# ---------------------------------------------------------------- 
# TODO theil_plot
# ---------------------------------------------------------------- 
- check the custom layout input
- write the docstring (input/output, support to save the plot)
- uniformize the plot colors (use the same as TriScale logo)
# --------------------------------------------

ValueError: Wrong file name extension. Valid extensions: 'png', 'jpg', 'svg', 'pdf'

### Performance Evaluation

In [153]:
data = [10,9,10,10,8,12,65,12,10]
KPI = {'percentile': 50,
       'confidence': 75,
       'class': 'one-sided',
       'side': 'upper'}
to_plot = ('vertical')
# to_plot = ('autocorr','horizontal')
verbose=True
triscale.analysis_kpi(data, KPI, to_plot, verbose)

# help(sort())

# ---------------------------------------------------------------- 
# TODO analysis_kpi 
# ---------------------------------------------------------------- 
- write the doctring
- work on the "default layout" for the plots (axis, title, written info)
# ---------------------------------------------------------------- 

Data appears i.i.d. (95%% confidence)

# ---------------------------------------------------------------- 
# TODO ThompsonCI 
# ---------------------------------------------------------------- 
- write the doctring
- check input types
- clean-up
# ---------------------------------------------------------------- 

With 9 data samples
x[2] and x[6] are the lower- and upper- bounds of 75%-confidence intervals for the median (one-sided).
# ---------------------------------------------------------------- 
# TODO ThompsonCI_plot 
# ---------------------------------------------------------------- 
- write the doctring
- check input types
- actually, we are more interested in (cu

(True, [2, 6])

### Repeatability

In [ ]:
data = [10,9,10,10,8,12,65,12,10]
confidence_repeatability = 99
to_plot = ('1D')
verbose=True

triscale.analysis_repeatability(data, confidence_repeatability, to_plot, verbose)

### Report

In [ ]:
meta_data={}

data_path = '../04_Pantheon/data/vegas/'
data_files = [
    'vegas_datalink_throughput_run1_flow1_egress.csv',
    'vegas_datalink_throughput_run1_flow1_ingress.csv',
#     'vegas_datalink_delay_run1_flow1.csv'
]
series = [data_path + file for file in data_files]
data = [series, series, series, series, series, series]
# print(data)

triscale.analysis_report(data, meta_data)

In [ ]:
data_path = '../04_Pantheon/data/'
perf_file = data_path + 'pantheon_perf.json'
perf = pantheon.parse_perf_file(perf_file)

data = [perf['bbr']['tput']]
# print(data)

series = np.array(perf['bbr']['tput'])
data = [series]
data_all = np.array(series)
for offset in [0.1,0.11,0]:
    series = series+offset
    data.append(series)
#     data_all=np.concatenate((data_all, series))


# etalon.analysis(perf['bbr']['delay'], 75, 75)
percentile = 25
confidence_percentile = 75    # in percent
confidence_repeatability = 95 # in percent
tolerance_repeatability= 0.15   # in percent
data_label='BBR Throughput'
triscale.analysis(data,
                percentile,
                confidence_percentile,
                bound_side = 'lower',
                confidence_repeatability = confidence_repeatability,
                tolerance_repeatability = tolerance_repeatability,
                data_label = data_label)

# help(etalon.analysis)
# help(theilslopes)

In [ ]:
from helpers import ci

data_path = '../04_Pantheon/data/'
perf_file = data_path + 'pantheon_perf.json'
perf = pantheon.parse_perf_file(perf_file)

series = np.array(perf['bbr']['tput'])
data = [series]
data_all = np.array(series)
for offset in [1,5,5,1,20]:
    series = series+offset
    data.append(series)
    data_all=np.concatenate((data_all, series))

# print(data_all)
data.append(data_all)
# len(data)
print(np.median(data_all))

ci( data, 80, 75, plot=True)

In [ ]:
# perf['bbr']['1']['all']

# perf.keys()

plot_title = 'local test in mahimahi, %s runs of %ss each per scheme' % (
                meta['run_times'],
                meta['runtime'])
data = []

for cc in perf.keys():
#     labels.append(cc)

    tputs  = []
    delays = []
    for run in perf[cc]:
        tputs.append(perf[cc][run]['all']['tput'])
        delays.append(perf[cc][run]['all']['delay'])
        
    trace = go.Scatter(
        x=delays,
        y=tputs,
        mode='markers',
        name=cc
                      )

    data.append(trace)

layout = go.Layout(
    title=plot_title,
    xaxis={
        'title':'95th percentile one-way delay (ms)',
        'autorange':'reversed'}, 
    yaxis={'title':'Average throughout (Mbit/s)'}, 
    width=800,
    height=600
)
figure=go.Figure(data=data,layout=layout)
iplot(figure)

In [ ]:
data_file = 'FlockLab_LinkTest_v1/2019-07_FlockLab_dpp-cc430.csv'
link_quality_bounds = [0,100]
link_quality_name = 'PRR [%]'
results = network_profiling(data_file, link_quality_bounds, link_quality_name)

results.head()
#results.dtypes

In [ ]:
data_file = 'FlockLab_LinkTest_v1/2019-07_FlockLab_sky.csv'
link_quality_bounds = [0,100]
link_quality_name = 'PRR [%]'
results = network_profiling(data_file, link_quality_bounds, link_quality_name)

results.head()
#results.dtypes

---
Anything below is leftovers from other notebooks
---
---

---
Anything below is leftovers from other notebooks
---
---

---
Anything below is leftovers from other notebooks
---
---

---
Anything below is leftovers from other notebooks
---
---

---
Anything below is leftovers from other notebooks
---
---

In [ ]:
pantheon.plot_ellipse(semimaj=5,semimin=1,phi=np.pi/3)


In [ ]:
# Create the sotring data frame
df = pd.DataFrame(columns=['test_number',
                          'date_time',
                          #'date',
                          #'time',
                          #'week',
                          #'weekday',
                          'B_n_slots',
                          'L_payload_size',
                          'R_random_seed',
                          'T_round',
                          'T_on_round',
                          'T_on_without_round',
                          'T_round_1slot'])

In [ ]:
# Set the path to the test results
serie = "serie1"
data_folder = Path(serie)
data_folder = data_folder / "results"
logs_folder = Path(serie)
logs_folder = logs_folder / "results_error_logs"

folder_list = [test_folder for test_folder in os.listdir(str(data_folder)) if os.path.isdir(os.path.join(str(data_folder), test_folder))]
for test_folder in sorted(folder_list):
    
    # Collect test information from summary
    g = open( str(data_folder / test_folder / "testsummary.csv"), "r")
    df_current = pd.read_csv( str(data_folder / test_folder / "testsummary.csv"), delimiter = ',')
    n_slots = df_current.at[0,"B_n_slots"]
    test_nb = df_current.at[0,"test_number"]
    g.close()
    
#     if test_nb != 67238:
#         continue
      
    # Add new test to the date frame
    df = pd.concat([df, df_current], sort=False)

    # Open the test serial log
    f = open( str(data_folder / test_folder / "serial.csv"), "r")
    
    test_result = np.zeros((len(node_lists), 4), dtype=np.float)
    node_index  = 0

    for node_id in node_lists:
        
        # reset file read offset 
        f.seek(0, 0)

        # tmp log
        nrg_log = []
        lat_log = []
        counter = 2* (n_slots + 1)  # number of lines with useful information to extract
        t_ref_error = 0             # abort if there might be an error of t_ref update
        sched_missed_counter = 0    # number of times the node missed the control packet
                                    # -> abort scanning after twice: (T,E) values may be wrong

        # read first line
        line = f.readline()
        while counter != 0 and line != '' and t_ref_error < 2 and sched_missed_counter < 2:
            if line[0]=='#':
                line = f.readline()
                continue

            tmp = line[0:-1].split(',')
            if int(tmp[2]) == node_id:
                
                if "sched rcv (1" in tmp[4]:
                    error_log = str(test_nb) + ' : Node ' + str(node_id) + ' bootstrapped on measured round'
                    error_log = error_log + '; discard it.'
                    print(error_log)
                    break
                    
                if "Missed 1 slots! Binary: 1" in tmp[4]:
                    t_ref_error += 1
                    
                if "Schedule missed or corrupted" in tmp[4]:
                    sched_missed_counter += 1
                    
                if "E: " in tmp[4]:
                    (trash, nrg) = tmp[4].split('E: ')
                    #print(nrg)
                    nrg_log.append(int(nrg))
                    counter -= 1

                if "T: " in tmp[4]:
                    (trash, lat) = tmp[4].split('T: ')
                    #print(line)
                    lat_log.append(int(lat))
                    counter -= 1

                #if "H " in tmp[4]:
                    #(trash, lat) = tmp[4].split('H ')
                    #print(tmp[4])
                    #lat_log.append(int(lat))
                    #counter -= 1

            line = f.readline()

        # Log test results
        if counter != 0:
            # Data is missing! Likely, this node failed to execute correctly
            # Discard all data from this node
            error_log = str(test_nb) + ' : Node ' + str(node_id) + ' failed'
            print(error_log)
            
            # Log the node serial output for inspection
            log_file = str(test_nb) + "_" + str(node_id)
            log_file = logs_folder / log_file
            string = str(node_id) + "," + str(node_id) + ","
            cmd = "sed '/" + string + "/!d' " + str(data_folder / test_folder / "serial.csv") + "> " + str(log_file)
            os.system(cmd)
            
            test_result[node_index][0] = np.nan
            test_result[node_index][1] = np.nan
            test_result[node_index][2] = np.nan
            test_result[node_index][3] = np.nan
        else:
            test_result[node_index][0] = nrg_log[0]         # E_w/
            test_result[node_index][1] = sum(nrg_log[1:])   # E_w/o
            test_result[node_index][2] = lat_log[0]         # T_n_slots
            test_result[node_index][3] = max(lat_log[1:])   # T_1

        # Increment the node index
        node_index += 1
    
#     print(test_result)
    
    # Compute the KPIs for the test
    T_on_round         = np.nanmedian(test_result,axis=0)[0]
    T_on_without_round = np.nanmedian(test_result,axis=0)[1]
    T_round            = np.nanmax(test_result,axis=0)[2]
    T_round_1slot      = np.nanmax(test_result,axis=0)[3]

    # Store them in df
    if np.isnan(T_round):
        print(str(test_nb) + ' completely failed!')
    else:
        df["T_round"].fillna(int(T_round), inplace = True)
        df["T_on_round"].fillna(int(T_on_round), inplace = True)
        df["T_on_without_round"].fillna(int(T_on_without_round), inplace = True)
        df["T_round_1slot"].fillna(int(T_round_1slot), inplace = True)
        
df.set_index('test_number', drop=True, inplace=True)
df

Now that we have all KPIs for our tests, we can do the systesis of the results, which leads to the performance report.

In [ ]:
df.groupby(['B_n_slots', 'L_payload_size']).max()

In [ ]:
hist = df.groupby(['B_n_slots', 'L_payload_size']).hist(bins=100)
hist

In [ ]:
df.loc[(df['B_n_slots'] == 5) & (df['L_payload_size'] == 64)].sort_values('T_round')
df.sort_values('date_time').head()


## 3. Repeatability test

Based on the results collected for our three experiments, we assess the repeatability of our results following the methodology presented in **INSERT REFERENCE**.

In [ ]:
# Repeatability test
# -> Look at existing implementations (scipy and other libraries)
# -> If none is available, write my own and push it somewhere.

confidence = 95 # in percentage

# Test
# ...
# Output
print("Assuming that the experimental conditions for the experiments were repeatable,")
print("the experimental results are repeatable with a " + str(confidence) + "% confidence.")

## 4. Aggregation and data vizualization

Since our experimental results have been tested as _repeatable_, we may aggregate the results from the different series into one single data set.

**Alternative**  
Since our experimental results have been tested as _non repeatable_, we must look at the individual experimental results (i.e., the results from each serie) in isolation.


In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

# Enable offline mode for plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


x = df.loc[(df['B_n_slots'] == 5) & (df['L_payload_size'] == 64)].sort_values('date_time').T_round.values.tolist()

cumsum = np.cumsum(sorted(x))
print(cumsum, cumsum[-1])
cumsum /= cumsum[-1]
print(cumsum, cumsum[-1])

trace = go.Scatter(x=sorted(x), 
                   y=cumsum,
                   mode='markers'
                  )
layout = go.Layout(
   title="Cumulative Distribution Function"
)
data = [trace]
# iplot(data)

trace = go.Histogram(x=sorted(x),
                     cumulative=dict(enabled=True))
data2 = [trace]
iplot(data2)

#print(sorted(x))

## 5. Performance report

In [ ]:
import scipy as sp
import math
from scipy import special,stats

def ci( data, percentile, confidence ):
    "This function computes the confidence interval for the given percentile of the data array, with the given confidence level"
    
    # Check validity of the inputs
    if confidence >= 100 or percentile <= 0:
        print("Invalid confidence: select a value strictly between 0 and 100")
        return
    if percentile >= 100 or percentile <= 0:
        print("Invalid percentile: select a value strictly between 0 and 100")
        return
    
    if percentile > 50:
        p_high = percentile
        percentile = 100 - percentile
        p_low = percentile
    else:
        p_low  = percentile
        p_high = 100-percentile
    
    # Make sure data in sorted
    data.sort()
    
    # get the index ranges
    N = len(data)
    Nmax = int(N/2)
    firstel = range(Nmax)
    lastel = [N-1-k for k in firstel]
    
    # compute all probabilities from the binomiale distribution for the percentile of interest
    bd=sp.stats.binom(N,percentile/100)
    ppm = [np.maximum(1-x,0.0) for x in np.cumsum([2*bd.pmf(k) for k in firstel])]

    # check the probabilities against the confidence level desired
    for k in reversed(range(Nmax)):
        if ppm[k]>=confidence/100:
            print('P_%g,%g: x_%d ... x_%d = %g ... %g' % (p_low, p_high, firstel[k]+1,lastel[k]+1,data[firstel[k]],data[lastel[k]]))
            return [ data[firstel[k]],data[lastel[k]] ]
    
    print('You do not have enough data to report a %.0f%s confidence interval.' % (confidence,'%'))
    return 

## 6. Final vizualization and outputs

In [ ]:
# trying to "extract" the usefull bit's from Hanspeter's code...

import scipy as sp
import sys,os,re
import math
from scipy import special,stats

# inputs
confidence = 75.0
percentile = 30.0
#data = df.groupby(['B_n_slots', 'L_payload_size'])

data = df.loc[(df['B_n_slots'] == 30) & (df['L_payload_size'] == 64)].sort_values('test_number').T_round.values
print(data)

In [ ]:
# get the index ranges
N = len(data)
Nmax = int(N/2)
firstel = range(Nmax)
lastel = [N-1-k for k in firstel]

print(N, firstel,lastel)

bd=sp.stats.binom(N,percentile/100)
ppm = [np.maximum(1-x,0.0) for x in np.cumsum([2*bd.pmf(k) for k in firstel])]


flag = 0
for k in reversed(range(Nmax)):
    if flag==0 and ppm[k]>=confidence/100:
        print('Percentile %g: x_%d ... x_%d = %g ... %g' % (percentile, firstel[k]+1,lastel[k]+1,data[firstel[k]],data[lastel[k]]))
        flag=1
if flag==0:
    print('You do not have enough data to report a %.0f%s confidence interval.' % (confidence,'%'))
    
    

In [ ]:
ci(data, 80, 95)

In [ ]:
# printing of the empirical cumulative distribution
# -> to be adapted

from plotsvg import Figure,White,Black

print('Your data drawn as in Fig.1')
fig=Figure()
fig.size=[5500,6400]
dmin=min(data)
dmax=max(data)
dd=dmax-dmin
xyrange=[[dmin-0.05*dd,dmax+0.05*dd],[0,100]]
fig.frame(xyrange, title='', xlabel='<i>x</i>', ylabel='Cumulative distribution', xscale='lin', yscale='prob', ticks='internal', xgrid=[], xgridtext=[], ygrid=[], ygridtext=[], sigmalines='#E00000')
fig.border_bgcolor=White
fig.frame_color=Black
fig.plotc(data)
print('<p><img src="table_paper_fig1.svg" alt="Data Plot" height="700"></p>')
fig.show(filename='%s/table_paper_fig1.svg' % dirname)